In [1]:
!nvidia-smi

Tue Dec 30 06:51:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
%%writefile merge_sort_gpu.cu

#include <iostream>                     // Для работы с вводом/выводом (cout, cin, endl)
#include <vector>                       // Подключение контейнера vector
#include <random>                       // Для генерации случайных чисел (random_device, mt19937, uniform_int_distribution)
#include <chrono>                       // измерение времени выполнения
#include <cuda_runtime.h>               // CUDA runtime API

using namespace std;                    // использование пространства имён std

#define THREADS 256                     // количество потоков в одном CUDA-блоке

// CUDA kernel: слияние подмассивов
__global__ void mergeKernel(int* input, int* output, int size, int width)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;   // глобальный индекс потока
    int start = tid * 2 * width;                       // начало подмассива

    if (start >= size) return;                         // проверка выхода за границы

    int mid = min(start + width, size);                // середина подмассива
    int end = min(start + 2 * width, size);            // конец подмассива

    int i = start;                                     // индекс левой части
    int j = mid;                                       // индекс правой части
    int k = start;                                     // индекс записи результата

    while (i < mid && j < end)                         // пока есть элементы в обеих частях
    {
        if (input[i] <= input[j])                      // сравнение элементов
            output[k++] = input[i++];                  // запись меньшего элемента
        else
            output[k++] = input[j++];                  // запись большего элемента
    }

    while (i < mid)                                    // если остались элементы слева
        output[k++] = input[i++];                      // копирование остатка

    while (j < end)                                    // если остались элементы справа
        output[k++] = input[j++];                      // копирование остатка
}

// Функция сортировки на GPU
void gpuMergeSort(int N)
{
    vector<int> h_data(N);                              // массив на CPU

    random_device rd;                                   // источник случайности
    mt19937 gen(rd());                                  // генератор случайных чисел
    uniform_int_distribution<> dist(0, 1000000);        // диапазон значений

    for (int i = 0; i < N; i++)                          // заполнение массива
        h_data[i] = dist(gen);                           // случайными числами

    int* d_in;                                          // указатель на входной массив GPU
    int* d_out;                                         // указатель на выходной массив GPU

    cudaMalloc(&d_in, N * sizeof(int));                 // выделение памяти на GPU
    cudaMalloc(&d_out, N * sizeof(int));                // выделение памяти на GPU

    cudaMemcpy(d_in, h_data.data(),                     // копирование данных
               N * sizeof(int),                         // размер данных
               cudaMemcpyHostToDevice);                 // с CPU на GPU

    auto start = chrono::high_resolution_clock::now();  // начало измерения времени

    for (int width = 1; width < N; width *= 2)          // многоступенчатое слияние
    {
        int blocks = (N + 2 * width - 1) / (2 * width); // расчёт числа блоков

        mergeKernel<<<blocks, THREADS>>>(               // запуск CUDA kernel
            d_in,                                       // входной массив
            d_out,                                      // выходной массив
            N,                                          // размер массива
            width);                                     // текущая ширина подмассивов

        cudaDeviceSynchronize();                         // ожидание завершения kernel

        int* temp = d_in;                               // временный указатель
        d_in = d_out;                                   // обмен массивов
        d_out = temp;                                   // для следующего шага
    }

    auto end = chrono::high_resolution_clock::now();    // конец измерения времени

    chrono::duration<double> time = end - start;        // вычисление времени выполнения

    cudaMemcpy(h_data.data(),                            // копирование результата
               d_in,                                     // из GPU
               N * sizeof(int),                          // размер данных
               cudaMemcpyDeviceToHost);                  // на CPU

    cout << "Размер массива: " << N << endl;            // вывод размера массива
    cout << "Время выполнения: " << time.count()        // вывод времени
         << " секунд" << endl;

    cudaFree(d_in);                                     // освобождение памяти GPU
    cudaFree(d_out);                                    // освобождение памяти GPU
}

// Главная функция
int main()
{
    cout << "Задача 4. Сортировка на GPU (CUDA)" << endl; // заголовок

    gpuMergeSort(10000);                                // сортировка массива 10 000
    gpuMergeSort(100000);                               // сортировка массива 100 000

    return 0;                                           // завершение программы
}



Overwriting merge_sort_gpu.cu


In [17]:
# Компиляция и запуск в Colab
!nvcc merge_sort_gpu.cu -o merge_sort_gpu
!./merge_sort_gpu


Задача 4. Сортировка на GPU (CUDA)
Размер массива: 10000
Время выполнения: 0.0113303 секунд
Размер массива: 100000
Время выполнения: 5.6577e-05 секунд
